In [55]:
%reset -f
import numpy as np
import pandas as pd
import os
import glob
from natsort import natsorted
import cv2
from utils import *
import sys
from tqdm import tqdm
import shutil

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [60]:
plate_files = ['30','32','35','38']
for ff in plate_files:
    DATADIR = results_dir(ff) # given a week number, select the data

    plates = natsorted(glob.glob(DATADIR+'*.JPG'))
    specs = natsorted(glob.glob(DATADIR+'*.txt'))
    path = '/home/kalfasyan/data/images/obj_detect_more_data/sticky_plate_imgs/'
    path_ann = path + 'annotations/'
    path_img = path + 'images/'
    class_map = pd.read_csv('./tmp/class_mapping.csv')

    for p, pname in tqdm(enumerate(plates)):

        pname = pname.split('/')[-1] # defining the platename
        if 'empty' in pname:
            continue
        plate_img = read_plate(pname, DATADIR) # reading the plate image
        spec = pd.read_csv(specs[p], sep="\t") # reading the specifications of the plate
        H,W,_ = plate_img.shape


        if p == 0: # fetching column names (only once needed)
            columns = [ii for ii in spec.columns if ii.endswith('.1')]
            colextensions = ['index', 'name plate', 'R','G','B']
            columns.extend(colextensions) # adding extra columns

        spec = spec[columns]
        spec.dropna(axis=0, how='any', inplace=True) # cleaning up

        # replacing commas with dots,
        tmp = [spec[col].str.replace(',','.').astype(float) for col in columns if col not in colextensions]
        spec = pd.concat(tmp, axis=1) # changing type to float
        del tmp

        # Adding insect classes columns in the dataframe and selecting/cleaning rows
        sub_map = class_map[class_map['fnames'] == pname[:-4]][['index','class']].set_index('index')
        spec['class'] = sub_map
        spec.drop(spec[spec['class']=='st'].index, inplace=True) # remove "stuk" class, which is not an insect
        spec = spec[spec['class'].astype('str').apply(lambda x: len(x.split('+'))) < 2] # remove classes with 2 insects
        spec.dropna(axis=0, how='any', inplace=True)

        # Adding yolo annotation format columns
        spec['yolo_class'] = 0
    #     xp = 30
        spec['yolo_x'] = np.abs(spec['Bounding Rect Right.1'] - np.abs(spec['Bounding Rect Left.1'] - spec['Bounding Rect Right.1']) /2) / W
        spec['yolo_y'] = np.abs(spec['Bounding Rect Bottom.1'] - np.abs(spec['Bounding Rect Top.1'] - spec['Bounding Rect Bottom.1']) /2) / H
        spec['yolo_width'] = np.abs(spec['Bounding Rect Left.1'] - spec['Bounding Rect Right.1']) / W
        spec['yolo_height'] = np.abs(spec['Bounding Rect Top.1'] - spec['Bounding Rect Bottom.1']) / H    

        spec = spec[(spec['yolo_height'].sort_values() > 0.007) & 
             (spec['yolo_width'].sort_values() > 0.007)]    


        ann_full_new = os.path.join( path_ann , "{}.txt".format(pname[:-4]) )
        img_full_new = os.path.join( path_img , pname.replace('JPG','jpg') )

        if not os.path.isfile( img_full_new ):
            print("Copying {}".format(pname[:-4]))
            cv2.imwrite(img_full_new, plate_img)
        if not len(spec) and not os.path.isfile( ann_full_new ):
            print('Empty file', ann_full_new)
            break
        elif not os.path.isfile( ann_full_new ):
            spec[['yolo_class',
                  'yolo_x',
                  'yolo_y',
                  'yolo_width',
                  'yolo_height']].to_csv(ann_full_new, 
                                         sep=' ', 
                                         index=False, 
                                         header=False)
        else:
            pass








0it [00:00, ?it/s]





1it [00:00,  3.93it/s]





2it [00:00,  4.03it/s]





3it [00:00,  4.00it/s]





4it [00:00,  4.05it/s]





5it [00:01,  4.07it/s]





6it [00:01,  4.13it/s]





7it [00:01,  4.15it/s]





8it [00:01,  4.18it/s]





9it [00:02,  4.21it/s]





10it [00:02,  4.38it/s]





11it [00:02,  4.47it/s]





12it [00:02,  4.55it/s]





13it [00:03,  4.46it/s]





14it [00:03,  4.39it/s]





15it [00:03,  4.36it/s]





16it [00:03,  4.27it/s]





17it [00:03,  4.26it/s]





18it [00:04,  4.24it/s]





19it [00:04,  4.41it/s]





20it [00:04,  4.53it/s]





21it [00:04,  4.45it/s]





22it [00:05,  4.38it/s]





23it [00:05,  4.35it/s]





24it [00:05,  4.32it/s]





25it [00:05,  4.27it/s]





26it [00:06,  4.26it/s]





27it [00:06,  4.26it/s]





28it [00:06,  4.41it/s]





29it [00:06,  4.35it/s]





30it [00:06,  4.30it/s]





31it [00:07,  4.27it/s]





32it [00:07,  4.21it/s]





33it [00:07,  4.22it/s]





34it [00:07,  4.18it/s